In [1]:
import pandas as pd
import numpy as np
import re
import os
from tqdm import tqdm
from geopy.geocoders import Nominatim
import mysql.connector
from sqlalchemy import create_engine
from datetime import datetime, timedelta

In [2]:
def save_data_to_disk(data, filename):
    if os.path.isfile(filename):
        existing_df = pd.read_csv(filename)
        data = pd.concat([existing_df, data], ignore_index=True)
        os.remove(filename)
        data.to_csv(filename,index=False)
    else:
        data.to_csv(filename, mode='w', header=True, index=False)

# loading the extracts columns to nomancleature

In [3]:
parquet_folder = "../completed extracts"
extracts = [i for i in os.listdir(parquet_folder) if 'Naukri_Extract' in i]
extracts.sort()

In [4]:
extracts

['Naukri_Extract_2024-04-19.parquet']

In [5]:
df = pd.DataFrame()
for i in extracts:
    i = parquet_folder+'/'+i
    temp = pd.read_parquet(i)
    print(temp.shape)
    df = pd.concat([df,temp],ignore_index=True)

(57035, 57)


In [6]:
df.columns = df.columns.str.replace(r'\W+', '_', regex=True).str.replace(r'_+', '_', regex=True).str.strip('_').str.lower()

In [7]:
df = df[~df.isna().all(axis=1)]

In [8]:
del df['']

In [9]:
df.rename(columns={'current_date':'extracted_on'},inplace=True)

In [10]:
df2 = pd.read_csv(parquet_folder+'/'+'PayPulse_backup_2024-04-07.csv')
df2 = df2[~df2['salary'].isin(['3-6', '6-10', '10-15', '15-25', '25-50', '50-75', '75-100','100-500', '0-3'])]
df2.rename(columns={
    'salmin':'salary_minimum',
    'salmax':'salary_maximum',
    'minexp':'experience_minimum', 
    'maxexp':'experience_maximum'
},inplace=True)

In [11]:
# def fillsal(row):
#     if 'not disclosed' in row['salary'].lower():
#         retval = '-'.join(row['sal_range'].split('to'))
#         return retval
#     else:
#         return row['salary']
# df['salary'] = df[['salary','sal_range']].apply(fillsal,axis=1)

# Correcting Salary Column

In [12]:
df = df[df['salary']!='Not Disclosed']
df = df[~df['salary'].str.contains('Not Disclosed')]
df = df[~df['salary'].str.contains('N/A')]

In [13]:
df.reset_index(drop=True,inplace=True)
del df['sal_range']

In [14]:
def extract_min_max(salary_str):
    if isinstance(salary_str, str):
        # print(salary_str)
        salary_str = salary_str.replace('₹ ','')
        if '$' in salary_str:
            salary_min = None
            salary_max = None 
            return pd.Series({'salmin': salary_min, 'salmax': salary_max})
        l = []
        salary_str = salary_str.replace('Less than', '')
        
        if '/month' in salary_str:
            salary_str = salary_str.replace('/month', '').replace(',', '')
            salary_min = None
            salary_max = None 
            return pd.Series({'salmin': salary_min, 'salmax': salary_max})
        
        try:
            a = salary_str.split('-')
            if a[0] == a[1]:        # min and max range are same numbers.
                salary_str = a[0]
        except:
            pass
            
        l = list(salary_str)
        
        if ('-' not in l) and all(ltr not in l for ltr in ['P','C','L']):
            salary_min = None
            salary_max = None
            return pd.Series({'salmin': salary_min, 'salmax': salary_max})
        
        if 'L' in l and 'C' in l:    
            ind = l.index('L')
            lak = ''.join(l[:ind])
            l = ''.join(l)
            l = l.replace(',','')
            if '-' in l:    
                l = l.split('-')
                indc = l[1].index('C')
                cro = ''.join(l[1][:indc])
                cro = cro.strip()
            if None in l:
                return pd.Series({'salmin': None, 'salmax': None})
            lak = lak.strip()
            if len(lak) > 4:
                lak = lak/100000 
            else:
                lak = float(lak)           
            cro = float(cro)
            cro = cro*100
            salary_min = lak 
            salary_max = cro  
            return pd.Series({'salmin': salary_min, 'salmax': salary_max})
        
        elif 'L' in l:    
            ind = l.index('L')
            l = ''.join(l[:ind])
            l = l.strip()
            l = [i.replace(',', '') for i in l]
            l = ''.join(l[:ind])
            if '-' in l:    
                l = l.split('-')
                if None in l:
                    return pd.Series({'salmin': None, 'salmax': None})
                l = [float(i)/100000 if len(i) > 4 else float(i) for i in l]
                salary_min = l[0]
                salary_max = l[1] 
                return pd.Series({'salmin': salary_min, 'salmax': salary_max})
            else:
                if len(l) > 4:
                    l = l/100000
                salary_min = l
                salary_max = l
                return pd.Series({'salmin': salary_min, 'salmax': salary_max})

        elif 'C' in l:
            ind = l.index('C')
            l = ''.join(l[:ind])
            l = l.strip()
            l = [i.replace(',', '') for i in l]
            l = ''.join(l[:ind])
            if '-' in l:    
                l = l.split('-')
                if None in l:
                    return pd.Series({'salmin': None, 'salmax': None})
                l = [float(i)/10000000 if len(i) > 4 else float(i)*100 for i in l]
                salary_min = l[0]
                salary_max = l[1] 
                return pd.Series({'salmin': salary_min, 'salmax': salary_max})
            else:
                salary_min = None
                salary_max = None
                return pd.Series({'salmin': salary_min, 'salmax': salary_max})

        elif 'P' in l:
            ind = l.index('P')
            l = l[:ind]
            l = ''.join(l)
            l = l.replace(',','').strip(' ')
            if '-' in l:
                l = l.split('-')
                salary_min = float(l[0])/100000
                salary_max = float(l[1])/100000
                return pd.Series({'salmin': salary_min, 'salmax': salary_max})
            else:
                salary_min = float(l)/100000
                salary_max = float(l)/100000
                return pd.Series({'salmin': salary_min, 'salmax': salary_max})
        
        elif ('-' in l):
            print(l)
            l = ''.join(l).strip()
            l = l.split('-')
            salary_min = l[0]
            salary_max = l[1]
            return pd.Series({'salmin': salary_min, 'salmax': salary_max})
        
        elif all(ltr not in l for ltr in ['P','C','L']):
            salary_min = None
            salary_max = None
            return pd.Series({'salmin': salary_min, 'salmax': salary_max})
    
    return pd.Series({'salmin': None, 'salmax': None})

df[['salary_minimum', 'salary_maximum']] = df['salary'].apply(extract_min_max)

In [15]:
# df['salmin'] = pd.to_numeric(df['salmin'],errors='coerce')
# df['salmax'] = pd.to_numeric(df['salmax'],errors='coerce')
df['salary_minimum'] = pd.to_numeric(df['salary_minimum'])
df['salary_maximum'] = pd.to_numeric(df['salary_maximum'])

In [16]:
for i in df[df['salary_minimum'].isna()]['salary'].unique(): # code skipped entries
    print(i)

In [17]:
df['salary_minimum'] = df['salary_minimum']*100000
df['salary_maximum'] = df['salary_maximum']*100000

In [18]:
df2['salary_minimum'] = df2['salary_minimum']*100000
df2['salary_maximum'] = df2['salary_maximum']*100000

# Correcting Loaction Columns

In [19]:
df['location'] = df['location'].str.lower()
df['location'] = df['location'].astype(str).str.lower().str.replace('hybrid -', '').str.strip()

In [20]:
df2['location'] = df2['location'].str.lower()
df2['location'] = df2['location'].astype(str).str.lower().str.replace('hybrid -', '').str.strip()

In [21]:
locset = set()
for loc in df['location']:
    sublocs = loc.split(',')
    for subloc in sublocs:
        locset.add(subloc.strip())
for loc in df2['location']:
    sublocs = loc.split(',')
    for subloc in sublocs:
        locset.add(subloc.strip())

In [22]:
len(locset)

2223

In [23]:
locationmapper = {}

In [24]:
anyinlist = ['delh','ncr','noida','guru','gurga']
allinlist = []
notinlist = []

apl = set()
for i in locset:
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl
locationmapper['Delhi']= apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['ahmed']
allinlist = []
notinlist = []

apl = set()
for i in locset:
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl
locationmapper['Ahmedabad']= apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['mumbai']
allinlist = []
notinlist = []

apl = set()
for i in locset:
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl
locationmapper['Mumbai']= apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['banga']
allinlist = []
notinlist = []

apl = set()
for i in locset:
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl
locationmapper['Bangalore']= apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['pune']
allinlist = []
notinlist = []

apl = set()
for i in locset:
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl
locationmapper['Pune']= apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['kolka']
allinlist = []
notinlist = []

apl = set()
for i in locset:
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl
locationmapper['Kolkata']= apl 
#------------------------------------------------------------------------------------------------------

In [25]:
"""Group Delhi – Delhi, Noida, Gurgaon, Gurugram 
Group Ahmedabad – Ahmedabad 
Group Mumbai - Mumbai, Navi Mumbai, Mumbai All areas 
Group Bangalore – Bangalore 
Group Pune – Pune 
Group Kolkata – Kolkata"""

'Group Delhi – Delhi, Noida, Gurgaon, Gurugram \nGroup Ahmedabad – Ahmedabad \nGroup Mumbai - Mumbai, Navi Mumbai, Mumbai All areas \nGroup Bangalore – Bangalore \nGroup Pune – Pune \nGroup Kolkata – Kolkata'

In [26]:
def location_mapper(lis):
    newlis = set()
    for i in range(len(lis)):
        for k,v in locationmapper.items():
            if lis[i] in v:
                newlis.add(k)
            # else:
                # newlis.add(lis[i])
    return ','.join(newlis)

In [27]:
for inde,row in df.iterrows():
    row = row['location'].split(',')
    df.at[inde,'location'] = location_mapper(row)

In [28]:
for inde,row in df2.iterrows():
    row = row['location'].split(',')
    df2.at[inde,'location'] = location_mapper(row)

# Correcting Experience Columns

In [29]:
wrong = []
for i in range(53):
    wrong.append(str(i)+' Yrs')
def correct_exp(exps):
    # print(exps)
    if pd.isna(exps) or exps == 'Nan' or exps == 'N/A':
        return np.nan
    exps = exps.replace('years','Yrs').replace('year','Yrs')
    # if "+" in exps:
    #     return '15+'
    if ('-' not in exps) and ('Yrs' in exps):#for 0 year(s) of experience
        try:
            exps = exps.split('Yrs')[0].strip().split(' ')[-1] + ' Yrs'
        except:
            pass    
    wrong1 = ['entry level']
    if exps in wrong1:
        exps = '0-2'    
    
    if exps in wrong:
        num = int(exps.split(' ')[0])
        exps = str(num)+'-'+str(num)

    if 'to' in exps: #From 5 to 10 year(s) of experience
        exp1 = exps.split('to')[0].strip().split(' ')[-1]
        exp2 = exps.split('to')[1].strip().split(' ')[0]
        exps = '-'.join([exp1,exp2])
    
    strang = str(exps[0])
    enrang = str(exps.split('-')[1].strip().split(' ')[0])
    exps = '-'.join([strang,enrang])
    return exps

df['dup_experience'] = df['experience'].apply(correct_exp)

In [30]:
df['experience_minimum'] = df['dup_experience'].apply(lambda x: float(x.split('-')[0].strip()) if pd.notnull(x) else None)
df['experience_maximum'] = df['dup_experience'].apply(lambda x: float(x.split('-')[1].strip().split(' ')[0]) if pd.notnull(x) else None)

In [31]:
del df['dup_experience']

# loading existing data

In [32]:
df = pd.concat([df,df2],ignore_index=True)

In [33]:
jobsdf = df[['job_title',
 'company_name',
 'experience',
 'salary',
 'salary_minimum',
 'salary_maximum',
 'experience_minimum',
 'experience_maximum',
 'posted_on',
 'extracted_on']]

In [34]:
# df2 = df2[['job_title', 'company_name', 'experience', 'salary', 'location','posted_on', 'extracted_on','role', 'industry_type', 'department', 'employment_type','role_category','skill1','skill2','skill3','skill4','skill5','skill6','skill7','skill8',
#      'salary_minimum', 'salary_maximum', 'experience_minimum', 'experience_maximum']]

In [35]:
# df = pd.concat([df,df2],ignore_index=True)

In [36]:
jobsdf.reset_index(drop=True,inplace=True)
jobsdf['id'] = jobsdf.index +1
df.reset_index(drop=True,inplace=True)
df['id'] = df.index +1

C:\Users\PM\AppData\Local\Temp\ipykernel_9776\1960682411.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobsdf['id'] = jobsdf.index +1


# Date_Correction

In [37]:
jobsdf['posted_on'] = jobsdf['posted_on'].str.replace('Posted: ','') 

C:\Users\PM\AppData\Local\Temp\ipykernel_9776\1438185700.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobsdf['posted_on'] = jobsdf['posted_on'].str.replace('Posted: ','')


In [38]:
jobsdf.loc[jobsdf['posted_on'].str.contains('Opening'),'posted_on']=jobsdf[jobsdf['posted_on'].str.contains('Opening')]['extracted_on']

In [39]:
def posted_corrector(value,ext):
    value = value.lower()
    wronglis = ['just now','few hours ago','today']
    c = value.replace('-','')
    if c.isdigit():
        return value
    if value not in wronglis:
        value = int(value.split(' ')[0].replace('+',''))
        if value < 100:
            date_obj = datetime.strptime(ext, "%Y-%m-%d")
            new_date = date_obj - timedelta(days=10)
            new_date_string = new_date.strftime("%Y-%m-%d")
            return new_date_string
        else:
            date_obj = datetime.strptime(ext, "%Y-%m-%d")
            return date_obj
    else:
        date_obj = datetime.strptime(ext, "%Y-%m-%d")
        return date_obj

In [40]:
jobsdf['posted_on'] = jobsdf.apply(lambda row: posted_corrector(row['posted_on'], row['extracted_on']), axis=1)

C:\Users\PM\AppData\Local\Temp\ipykernel_9776\1815589175.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobsdf['posted_on'] = jobsdf.apply(lambda row: posted_corrector(row['posted_on'], row['extracted_on']), axis=1)


In [41]:
jobsdf['posted_on'] = pd.to_datetime(jobsdf['posted_on']).dt.date

C:\Users\PM\AppData\Local\Temp\ipykernel_9776\4244471813.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobsdf['posted_on'] = pd.to_datetime(jobsdf['posted_on']).dt.date


# Creating Tables and push to Database

In [42]:
# employment_type
# mapped_average_sal
# mapped_job_title
# combined_skills
# ori_experience
# title_id

In [43]:
del df['combined_skills']

In [45]:
df.to_parquet('Cleaned_Extract.parquet',index=False)

In [168]:
jobsdf.reset_index(drop=True,inplace=True)
jobsdf['id'] = jobsdf.index +1

C:\Users\PM\AppData\Local\Temp\ipykernel_12692\2504277871.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobsdf['id'] = jobsdf.index +1


In [169]:
# del df['id']

In [170]:
# del df['combined_skills']

In [171]:
# host = "164.52.194.109"
# user = "root"
# password = "equipaypartners"
# database = "paypulse"

# engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

# df.to_sql(name='naukri_staging', con=engine, if_exists='replace', index=False)

In [172]:
jobsdf

,job_title,company_name,experience,salary,salary_minimum,salary_maximum,experience_minimum,experience_maximum,posted_on,extracted_on,id
0,Customer Service Executive - Up To 32k: Fluent...,Wipro,0 - 5 years,2.25-4 Lacs P.A.,225000.0,400000.0,0.0,5.0,2024-04-06,2024-04-16,1
1,AppDynamics,Tata Consultancy Services (TCS),4 - 6 years,5.5-15 Lacs P.A.,550000.0,1500000.0,4.0,6.0,2024-04-06,2024-04-16,2
2,Sales Executive-Building Segment,Fosroc Chemicals,2 - 7 years,4-6 Lacs P.A.,400000.0,600000.0,2.0,7.0,2024-04-06,2024-04-16,3
3,Mainframe Developer,Tata Consultancy Services (TCS),4 - 9 years,"60,000-3 Lacs P.A.",60000.0,300000.0,4.0,9.0,2024-04-06,2024-04-16,4
4,Admission Counsellor,upGrad,1 - 6 years,4-9 Lacs P.A.,400000.0,900000.0,1.0,6.0,2024-04-07,2024-04-17,5
...,...,...,...,...,...,...,...,...,...,...,...
120184,Executive-HR,jbm neel metal,3 - 8 Yrs,2.25-4.5 Lacs P.A.,225000.0,450000.0,3.0,8.0,2024-03-25,2024-04-04,120185
120185,Engineer-Robot Maintenance,jbm neel metal,3 - 8 Yrs,2.5-5 Lacs P.A.,250000.0,500000.0,3.0,8.0,2024-03-25,2024-04-04,120186
120186,Executive-Quality,neel metal products ltd,1 - 4 Yrs,1-3.5 Lacs P.A.,100000.0,350000.0,1.0,4.0,2024-03-25,2024-04-04,120187
120187,Payment Associate(Customer Support Executive,kochar infotech,0-2,2-2.5 Lacs P.A.,200000.0,250000.0,0.0,2.0,2024-03-25,2024-04-04,120188


## location tables

In [173]:
df['location'] = df['location'].astype('str')

In [174]:
unique_locations = set()
for ind,row in df.iterrows():
    if isinstance(row['location'],str):
        loclist = row['location'].split(',')
        for i in loclist:
            unique_locations.add(i)

In [175]:
locationdf = pd.DataFrame(unique_locations,columns=['location'])
locationdf = locationdf[locationdf['location'] != '']
locationdf.reset_index(drop=True,inplace=True)
locationdf['id'] = locationdf.index+1

In [176]:
joblocation = []
for locind, locrow in locationdf.iterrows():
    for dfind, dfrow in df[df['location'].str.contains(re.escape(locrow['location']))].iterrows():
        joblocation.append({'location_id': locrow['id'], 'job_id': dfrow['id']})

In [177]:
joblocationdf = pd.DataFrame(joblocation)
joblocationdf['id'] = joblocationdf.index+1

In [178]:
joblocationdf

,location_id,job_id,id
0,1,2,1
1,1,5,2
2,1,10,3
3,1,18,4
4,1,23,5
...,...,...,...
76748,6,120147,76749
76749,6,120174,76750
76750,6,120181,76751
76751,6,120182,76752


## Skill Tables

In [179]:
skill_columns = df.filter(like='skill').columns
skillset = set(df[skill_columns].values.flatten())

In [180]:
skilldf = pd.DataFrame(skillset,columns=['skill'])
skilldf.dropna(inplace=True)
skilldf['id'] = skilldf.index+1

In [181]:
df['combined_skills'] = df[skill_columns].apply(lambda row: ','.join(row.dropna().astype(str)), axis=1)

In [182]:
df['combined_skills'] = df['combined_skills'].astype('str')

In [183]:
jobskill = []
for skillind, skillrow in skilldf.iterrows():
    for dfind, dfrow in df[df['combined_skills'].str.contains(re.escape(skillrow['skill']))].iterrows():
        jobskill.append({'skill_id': skillrow['id'], 'job_id': dfrow['id']})

In [184]:
jobskilldf = pd.DataFrame(jobskill)

In [185]:
jobskilldf['id'] = jobskilldf.index+1

In [186]:
jobskilldf

,skill_id,job_id,id
0,1,5748,1
1,2,4108,2
2,2,4942,3
3,3,8013,4
4,4,7017,5
...,...,...,...
1449351,16619,1422,1449352
1449352,16620,4582,1449353
1449353,16621,2133,1449354
1449354,16621,4277,1449355


## industry table

In [187]:
countkey = 500

In [188]:
df['industry_type'] = df['industry_type'].str.lower()

In [189]:
industrydf = df['industry_type'].value_counts().reset_index()
industrydf.columns = ['industry', 'count']
industrydf = industrydf[(industrydf['count'] > countkey)]
industrydf['id']=industrydf.index+1

In [190]:
del industrydf['count']

## Department table

In [191]:
df['department'] = df['department'].str.lower()

In [192]:
departmentdf = df['department'].value_counts().reset_index()
departmentdf.columns = ['department', 'count']
departmentdf = departmentdf[(departmentdf['count'] > countkey)]
departmentdf['id']=departmentdf.index+1

In [193]:
del departmentdf['count']

## rolecategory

In [194]:
df['role_category'] = df['role_category'].str.lower()

In [195]:
rolecategorydf = df['role_category'].value_counts().reset_index()
rolecategorydf.columns = ['role_category', 'count']
rolecategorydf = rolecategorydf[(rolecategorydf['count'] > countkey)]
rolecategorydf['id']=rolecategorydf.index+1

In [196]:
del rolecategorydf['count']

## role table

In [197]:
df['role'] = df['role'].str.lower()

In [198]:
roledf = df['role'].value_counts().reset_index()
roledf.columns = ['role', 'count']
roledf = roledf[(roledf['count'] > countkey)]
roledf['id']=roledf.index+1

In [199]:
del roledf['count']

In [200]:
"""industry_id
department_id
role_category_id
role_id"""

'industry_id\ndepartment_id\nrole_category_id\nrole_id'

## all ids in job table

In [201]:
# for jobinde,jobrow in joblocationdf.iterrows():
#     jobsdf.at[jobrow['job_id'],'location_id'] = jobrow['location_id']
    
# for jobinde,jobrow in jobskilldf.iterrows():
#     jobsdf.at[jobrow['job_id'],'skill_id'] = jobrow['skill_id']

In [202]:
for jobinde,jobrow in industrydf.iterrows():
    indexes = df.loc[df['industry_type'] == jobrow['industry']].index
    jobsdf.loc[indexes,'industry_id'] = jobrow['id']
    
for jobinde,jobrow in departmentdf.iterrows():
    indexes = df.loc[df['department'] == jobrow['department']].index
    jobsdf.loc[indexes,'department_id'] = jobrow['id']    
    
for jobinde,jobrow in rolecategorydf.iterrows():
    indexes = df.loc[df['role_category'] == jobrow['role_category']].index
    jobsdf.loc[indexes,'role_category_id'] = jobrow['id']    
    
for jobinde,jobrow in roledf.iterrows():
    indexes = df.loc[df['role'] == jobrow['role']].index
    jobsdf.loc[indexes,'role_id'] = jobrow['id']            

C:\Users\PM\AppData\Local\Temp\ipykernel_12692\3360376506.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobsdf.loc[indexes,'industry_id'] = jobrow['id']


C:\Users\PM\AppData\Local\Temp\ipykernel_12692\3360376506.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobsdf.loc[indexes,'department_id'] = jobrow['id']
C:\Users\PM\AppData\Local\Temp\ipykernel_12692\3360376506.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobsdf.loc[indexes,'role_category_id'] = jobrow['id']
C:\Users\PM\AppData\Local\Temp\ipykernel_12692\3360376506.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [203]:
jobsdf

,job_title,company_name,experience,salary,salary_minimum,salary_maximum,experience_minimum,experience_maximum,posted_on,extracted_on,id,industry_id,department_id,role_category_id,role_id
0,Customer Service Executive - Up To 32k: Fluent...,Wipro,0 - 5 years,2.25-4 Lacs P.A.,225000.0,400000.0,0.0,5.0,2024-04-06,2024-04-16,1,3.0,10.0,30.0,NaN
1,AppDynamics,Tata Consultancy Services (TCS),4 - 6 years,5.5-15 Lacs P.A.,550000.0,1500000.0,4.0,6.0,2024-04-06,2024-04-16,2,2.0,5.0,11.0,20.0
2,Sales Executive-Building Segment,Fosroc Chemicals,2 - 7 years,4-6 Lacs P.A.,400000.0,600000.0,2.0,7.0,2024-04-06,2024-04-16,3,13.0,2.0,3.0,16.0
3,Mainframe Developer,Tata Consultancy Services (TCS),4 - 9 years,"60,000-3 Lacs P.A.",60000.0,300000.0,4.0,9.0,2024-04-06,2024-04-16,4,2.0,3.0,2.0,3.0
4,Admission Counsellor,upGrad,1 - 6 years,4-9 Lacs P.A.,400000.0,900000.0,1.0,6.0,2024-04-07,2024-04-17,5,18.0,2.0,4.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120184,Executive-HR,jbm neel metal,3 - 8 Yrs,2.25-4.5 Lacs P.A.,225000.0,450000.0,3.0,8.0,2024-03-25,2024-04-04,120185,30.0,6.0,16.0,NaN
120185,Engineer-Robot Maintenance,jbm neel metal,3 - 8 Yrs,2.5-5 Lacs P.A.,250000.0,500000.0,3.0,8.0,2024-03-25,2024-04-04,120186,30.0,9.0,19.0,NaN
120186,Executive-Quality,neel metal products ltd,1 - 4 Yrs,1-3.5 Lacs P.A.,100000.0,350000.0,1.0,4.0,2024-03-25,2024-04-04,120187,30.0,NaN,NaN,NaN
120187,Payment Associate(Customer Support Executive,kochar infotech,0-2,2-2.5 Lacs P.A.,200000.0,250000.0,0.0,2.0,2024-03-25,2024-04-04,120188,3.0,10.0,NaN,NaN


## tables to sql

In [204]:
host = "164.52.194.109"
user = "root"
password = "equipaypartners"
database = "paypulse"

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

In [205]:
skilldf

,skill,id
0,SVA,1
1,Stat Audit,2
2,Arc Map,3
3,eee,4
4,Ercp,5
...,...,...
16616,TCH RTP,16617
16617,Contracts,16618
16618,Microsoft Dynamics NAV,16619
16619,PI Perfmon,16620


In [206]:
industrydf.to_sql(name='industry', con=engine, if_exists='append', index=False)
departmentdf.to_sql(name='department', con=engine, if_exists='append', index=False)
rolecategorydf.to_sql(name='role_category', con=engine, if_exists='append', index=False)
roledf.to_sql(name='role', con=engine, if_exists='append', index=False)
locationdf.to_sql(name='location', con=engine, if_exists='append', index=False)
skilldf.to_sql(name='skill', con=engine, if_exists='append', index=False)

16619

In [207]:
jobsdf.to_sql(name='job', con=engine, if_exists='append', index=False)

120189

In [208]:
joblocationdf.to_sql(name='job_location', con=engine, if_exists='append', index=False)
jobskilldf.to_sql(name='job_skill', con=engine, if_exists='append', index=False)

1449356

# End